In [4]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar_template as lc
from tensorflow.contrib.layers import flatten
import matplotlib.pyplot as plt
import time

In [5]:
#fetch the data

def fetch_data(directory, label): 
    item_list = []
    step = 0
    for file in tqdm(os.listdir(directory)):
        full_img_str = directory + "/" + file
        #print(full_img_str)

        mat = spio.loadmat(full_img_str, squeeze_me=True)
        data = np.abs(mat["data_store"])
        smaller_data = data[100:400]
        
        image_resized = resize(smaller_data, (100, 320, 4),
                       anti_aliasing=True)
        #print(data.shape)
        #print(data)
        ###append the img and label to the list###
        sub_list = [image_resized, label]
        #print(sub_list)
        item_list.append(sub_list)
        
    return item_list
    
bike1 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1000/data", [1, 0, 0])
bike2 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1001/data", [1, 0, 0])
bike3 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1002/data", [1, 0, 0])

car1 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1000/data", [0, 1, 0])
car2 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1001/data", [0, 1, 0])
car3 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1002/data", [0, 1, 0])

ped1 = fetch_data("Desktop/ee596prepro/2019_04_09_pms1000/data", [0, 0, 1])
ped2 = fetch_data("Desktop/ee596prepro/2019_04_09_pms1001/data", [0, 0, 1])
ped3 = fetch_data("Desktop/ee596prepro/2019_04_09_pms2000/data", [0, 0, 1])



#directory = "Desktop/ee596prepro/2019_04_09_bms1000/data"

In [ ]:
def mini_batch(features,labels,mini_batch_size):
    """
    Args:
        features: features for one batch
        labels: labels for one batch
        mini_batch_size: the mini-batch size you want to use.
    Hint: Use "yield" to generate mini-batch features and labels
    """
    #split the data into batches
    amount_of_data = len(features)
    number_of_bunches = amount_of_data/mini_batch_size
    
    bunches_features = []
    bunches_labels = []
    
    #loop over breaking the data into batches
    for i in range(int(number_of_bunches)):
        current_range = i * mini_batch_size
        f_b = features[current_range:current_range+mini_batch_size]
        l_b = labels[current_range:current_range+mini_batch_size]
        
        bunches_features.append(f_b)
        bunches_labels.append(l_b)
    
    #return the mini-batched data
    return bunches_features, bunches_labels

#full_list = bike1 + bike2 + bike3 + car1 + car2 + car3 + ped1 + ped2 + ped3
full_list = bike1 + car1 + ped1 + bike2 + car2 + ped2# + bike3 + car3 + ped3
test_list = bike3 + car3 + ped3

np.random.shuffle(full_list)
np.random.shuffle(test_list)

training_set = full_list#[0:2000]
valid_set = test_list[0:500]
test_set = test_list[500:997]


train_set_data = []
train_set_labels = []
valid_set_data = []
valid_set_labels = []
test_set_data = []
test_set_labels = []
#print(np.asarray(training_set).shape)
#print(training_set[0][0][0])
#print(training_set[0][:][1])
#print(training_set[0][:][0].shape)
#print(training_set[0][:][0])
#split into training, valid, and testing
for i in range(len(training_set)):
    train_set_data.append(training_set[i][0])
    train_set_labels.append(training_set[i][1])
    
for i in range(len(valid_set)):
    valid_set_data.append(valid_set[i][0])
    valid_set_labels.append(valid_set[i][1])

for i in range(len(test_set)):
    test_set_data.append(test_set[i][0])
    test_set_labels.append(test_set[i][1])

#print(np.asarray(train_set_data).shape)
#print(train_set_data.shape)


train_set_data, train_set_labels = mini_batch(train_set_data,train_set_labels,5)
valid_set_data, valid_set_labels = mini_batch(valid_set_data,valid_set_labels,5)
test_set_data, test_set_labels = mini_batch(test_set_data,test_set_labels,5)

#training_set = None
#valid_set = None
#test_set = None
#full_list = None
#bike1 = None
#car1 = None
#ped1 = None
#test_list = None

In [11]:
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#import keras.models
#import keras.layers

#X = seq.reshape(len(seq), 1, 1)
#y = seq.reshape(len(seq), 1)
seq_imgs = 25
print(np.asarray(train_set_data).shape)
#train_set_data = np.reshape(train_set_data, (980*25, 227, 227, 3))
#print(train_set_labels.shape)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(2, (5, 5)),
                              input_shape=(seq_imgs, 227, 227, 3)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D(2, (2, 2))))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
model.add(tf.keras.layers.LSTM(50))
model.add(tf.keras.layers.Dense(2))


model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(train_set_data, train_set_labels, epochs=20, batch_size=128)



(980, 25, 227, 227, 3)


ValueError: Error when checking input: expected time_distributed_3_input to have 5 dimensions, but got array with shape (24500, 227, 227, 3)